In [1]:
import json
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

/Users/danilobabic/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
folder_path = '/Users/danilobabic/Documents/GitHub/ri_train_data'
words = []
missing_videos = []
video_metadata = []
words_file = '/Users/danilobabic/Documents/GitHub/archive/wlasl_class_list.txt'
missing_videos_file = '/Users/danilobabic/Documents/GitHub/archive/missing.txt'
video_metadata_file = '/Users/danilobabic/Documents/GitHub/archive/WLASL_v0.3.json'
archive_path = '/Users/danilobabic/Documents/GitHub/archive/videos'
testing_path = '/Users/danilobabic/Documents/GitHub/testing'
mp_holistic = mp.solutions.holistic

In [3]:
def get_words(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip().split('\t')
            words.append(line[1])

In [4]:
get_words(words_file)


In [5]:
def get_missing_files(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            missing_videos.append(line)

In [6]:
get_missing_files(missing_videos_file)

In [7]:
def get_video_metadata(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        return data

In [8]:
video_metadata = get_video_metadata(video_metadata_file)

In [9]:
def create_word_folders(path):
    for word in words:
        folder_path = os.path.join(path, word)
        os.makedirs(folder_path, exist_ok=True)
    

In [10]:
def extract_keypoints(frame, model):         
    results = model.process(frame)                 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [11]:
def extract_videos():
    #print("USO U FUNKCIJUUUUU")
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        print(video_metadata)
        # for each word we find videos 
        for word in video_metadata:
            for video in word['instances']:
                if video['video_id'] in missing_videos or video['split'] != 'train': 
                    continue
                video_path = os.path.join(folder_path, word['gloss'], video['video_id'])
                os.makedirs(video_path, exist_ok=True)
                cap = cv2.VideoCapture(archive_path + "/" + video['video_id'] + ".mp4")
                print("USO U FUNCKIJU FAJL :" + archive_path + "/" + video['video_id'] + ".mp4")
                if video['frame_end'] == -1:
                    frame_num = 1
                    while True: 
                        ret, frame = cap.read()
                        if not ret:
                            break
                        keypoints = extract_keypoints(frame, holistic)
                        npy_path = os.path.join(video_path, str(frame_num))
                        np.save(npy_path, keypoints)
                        frame_num += 1
                else:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, video['frame_start'])
                    frame_num = video['frame_start']
                    while frame_num <= video['frame_end']:
                        ret, frame = cap.read()
                        if not ret:
                            break 
                        keypoints = extract_keypoints(frame, holistic)
                        npy_path = os.path.join(video_path, str(frame_num))
                        np.save(npy_path, keypoints)
                        frame_num += 1 
                        if frame_num > video['frame_end']:
                            break

In [12]:
def load_data_from_folders():
    frames = []
    labels = []
    word_folders = sorted([f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and f != '.DS_Store'])
    #testing_list = ["book","minute","cherry"]
    for word_folder in word_folders:
        word_path = os.path.join(folder_path, word_folder)
        video_folders = sorted([f for f in os.listdir(word_path) if os.path.isdir(os.path.join(word_path, f)) and f != '.DS_Store'])
        for video_folder in video_folders:
            video_path = os.path.join(word_path, video_folder)
            frame_files = sorted(os.listdir(video_path))
            video_frames = []
            for frame_file in frame_files:
                frame_path = os.path.join(video_path, frame_file)
                frame = np.load(frame_path)
                video_frames.append(frame)
            frames.append(video_frames)
            labels.append(label_map[word_folder])
    return frames, labels

In [13]:
label_map = {label:num for num, label in enumerate(words)}

In [14]:
frames, labels = load_data_from_folders()
#print(frames)
max_frame_length = max(len(frame) for frame in frames)
#print(labels)
# Pad the frames to have a common length
#padded_frames = tf.keras.preprocessing.sequence.pad_sequences(frames, maxlen=max_frame_length, padding='post', value=0.0)
frames = tf.keras.preprocessing.sequence.pad_sequences(frames, padding='post', dtype='float32')
labels = tf.keras.utils.to_categorical(labels)
# Convert the padded_frames to a NumPy array
frames_array = np.array(frames)
labels_array = np.array(labels)
print(frames_array.shape)
print(labels_array.shape)
#print(labels_array)


(8313, 233, 1662)
(8313, 2000)


In [15]:
def create_lstm_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(64, return_sequences=True, activation='relu', input_shape=(None,1662)))
    model.add(tf.keras.layers.LSTM(128, return_sequences=True, activation='relu'))
    model.add(tf.keras.layers.LSTM(64, return_sequences=False, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(labels_array.shape[1], activation='softmax'))
    return model

In [16]:
log_dir = os.path.join('Logs')
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [17]:
model = create_lstm_model()
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(frames_array, labels_array, epochs= 10, callbacks=[tb_callback])
model.summary()

Epoch 1/10
260/260 [==============================] - 131s 486ms/step - loss: nan - categorical_accuracy: 4.8117e-04
Epoch 2/10
260/260 [==============================] - 129s 495ms/step - loss: nan - categorical_accuracy: 6.0147e-04
Epoch 3/10
260/260 [==============================] - 128s 494ms/step - loss: nan - categorical_accuracy: 6.0147e-04
Epoch 4/10
260/260 [==============================] - 128s 493ms/step - loss: nan - categorical_accuracy: 6.0147e-04
Epoch 5/10
260/260 [==============================] - 129s 494ms/step - loss: nan - categorical_accuracy: 6.0147e-04
Epoch 6/10
260/260 [==============================] - 126s 486ms/step - loss: nan - categorical_accuracy: 6.0147e-04
Epoch 7/10
260/260 [==============================] - 129s 495ms/step - loss: nan - categorical_accuracy: 6.0147e-04
Epoch 8/10
260/260 [==============================] - 129s 497ms/step - loss: nan - categorical_accuracy: 6.0147e-04
Epoch 9/10
260/260 [==============================] - 127s 488ms

In [18]:
#create_word_folders(testing_path)  #folder for training data

In [19]:
def extract_test_videos():
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        print(video_metadata)
        # for each word we find videos 
        for word in video_metadata:
            if word['gloss'] in ["book","minute","cherry"]:
                continue
            for video in word['instances']:
                if video['video_id'] in missing_videos or video['split'] != 'test': 
                    continue
                video_path = os.path.join(testing_path, word['gloss'], video['video_id'])
                os.makedirs(video_path, exist_ok=True)
                cap = cv2.VideoCapture(archive_path + "/" + video['video_id'] + ".mp4")
                print("USO U FUNCKIJU FAJL :" + archive_path + "/" + video['video_id'] + ".mp4")
                if video['frame_end'] == -1:
                    frame_num = 1
                    while True: 
                        ret, frame = cap.read()
                        if not ret:
                            break
                        keypoints = extract_keypoints(frame, holistic)
                        npy_path = os.path.join(video_path, str(frame_num))
                        np.save(npy_path, keypoints)
                        frame_num += 1
                else:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, video['frame_start'])
                    frame_num = video['frame_start']
                    while frame_num <= video['frame_end']:
                        ret, frame = cap.read()
                        if not ret:
                            break 
                        keypoints = extract_keypoints(frame, holistic)
                        npy_path = os.path.join(video_path, str(frame_num))
                        np.save(npy_path, keypoints)
                        frame_num += 1 
                        if frame_num > video['frame_end']:
                            break

In [20]:
#extract_test_videos()

In [21]:
# FOR TESTING
def load_data_from_folders_test(): 
    frames = []
    labels = []
    word_folders = sorted([f for f in os.listdir(testing_path) if os.path.isdir(os.path.join(testing_path, f)) and f != '.DS_Store'])
    #testing_list = ["book","minute","cherry"]
    for word_folder in word_folders:
        word_path = os.path.join(testing_path, word_folder)
        video_folders = sorted([f for f in os.listdir(word_path) if os.path.isdir(os.path.join(word_path, f)) and f != '.DS_Store'])
        for video_folder in video_folders:
            video_path = os.path.join(word_path, video_folder)
            frame_files = sorted(os.listdir(video_path))
            video_frames = []
            for frame_file in frame_files:
                frame_path = os.path.join(video_path, frame_file)
                frame = np.load(frame_path)
                video_frames.append(frame)
            frames.append(video_frames)
            labels.append(label_map[word_folder])
    return frames, labels

In [ ]:
frames_test, labels_test = load_data_from_folders_test()
print(labels)
frames_test = tf.keras.preprocessing.sequence.pad_sequences(frames_test, padding='post', dtype='float32')
labels_test = tf.keras.utils.to_categorical(labels_test)
# Convert the padded_frames to a NumPy array
frames_array_test = np.array(frames_test)
labels_array_test = np.array(labels_test)
print(frames_array_test.shape)
print(labels_array_test.shape)
#print(labels_array)

In [ ]:
words_predicted = model.predict(frames_array_test)
multilabel_confusion_matrix(np.argmax(labels_array_test, axis=1), np.argmax(words_predicted , axis=1))
accuracy_score(np.argmax(labels_array_test, axis=1), np.argmax(words_predicted , axis=1))

In [ ]:
import matplotlib.pyplot as plt
history = model.history.history
accuracy = history['categorical_accuracy']
val_accuracy = history['val_categorical_accuracy']
epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()